# Imports 

In [13]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Get the data

In [14]:
os.chdir(r"C:\Users\Áron\Google Drive\UNI\Y4S2\Dissertation\Data")
CHOSEN_SHEETS = ["England", "Scotland", "Wales", "Northern Ireland"]

covid_data = pd.read_excel(r"summary_data_2018_2019.xlsx", skiprows = 5, nrows = 37, sheet_name = CHOSEN_SHEETS) #skip nhs placement questions
covid_data

{'England':                                            Unnamed: 0  % agree  Responses  \
 0                           The teaching on my course      NaN        NaN   
 1            1 - Staff are good at explaining things.     89.0   251764.0   
 2        2 - Staff have made the subject interesting.     82.0   251586.0   
 3       3 - The course is intellectually stimulating.     85.0   251659.0   
 4   4 - My course has challenged me to achieve my ...     81.0   251723.0   
 5                              Learning opportunities      NaN        NaN   
 6   5 - My course has provided me with opportuniti...     84.0   251730.0   
 7   6 - My course has provided me with opportuniti...     85.0   251561.0   
 8   7 - My course has provided me with opportuniti...     81.0   251519.0   
 9                             Assessment and feedback      NaN        NaN   
 10  8 - The criteria used in marking have been cle...     72.0   251630.0   
 11          9 - Marking and assessment has been fair

In [15]:
covid_data.get("England")

,Unnamed: 0,% agree,Responses,% agree.1,Responses.1,% agree.2,Responses.2,% agree.3,Responses.3
0,The teaching on my course,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1 - Staff are good at explaining things.,89.0,251764.0,86.0,19630.0,89.0,259777.0,86.0,21234.0
2,2 - Staff have made the subject interesting.,82.0,251586.0,82.0,19661.0,82.0,259593.0,80.0,21227.0
3,3 - The course is intellectually stimulating.,85.0,251659.0,89.0,20002.0,84.0,259704.0,88.0,21553.0
4,4 - My course has challenged me to achieve my ...,81.0,251723.0,84.0,19993.0,81.0,259749.0,84.0,21536.0
5,Learning opportunities,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,5 - My course has provided me with opportuniti...,84.0,251730.0,85.0,19980.0,84.0,259722.0,85.0,21538.0
7,6 - My course has provided me with opportuniti...,85.0,251561.0,84.0,19931.0,85.0,259638.0,83.0,21501.0
8,7 - My course has provided me with opportuniti...,81.0,251519.0,79.0,19792.0,81.0,259525.0,80.0,21361.0
9,Assessment and feedback,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Cleaning

In [16]:
def weighted_avg(file, idx):
    return (file.iloc[idx][1] * file.iloc[idx][2] + file.iloc[idx][3] * file.iloc[idx][4]) / (file.iloc[idx][2]+ file.iloc[idx][4])

def clean(file, sheet_name):
    df = file.get(sheet_name)
    df.dropna(axis = "rows", inplace = True) #drop empty rows - just text
    df.reset_index(inplace = True, drop = True) #reset index

    average_pct = []
    response_counts = []
    for i in range(len(df)):
        average_pct.append(weighted_avg(df, i))
        response_counts.append(df.iloc[i][2] + df.iloc[i][4])
    
        
    df.drop(df.columns[[1,2,3,4,5,6,7,8]], axis=1, inplace=True) #drop columns, no longer needed
    df["score"] = average_pct #add score as feature
    
    df = df.T #set transpose as we want Qs af features
    df.rename(columns= lambda x: "Q"+str(x+1), inplace = True) #rename
    df.drop("Unnamed: 0", inplace = True, axis = "rows") #drop question row, now in header
    df.rename(index = {"score": sheet_name}, inplace = True) #keep track of rows
    df["avg_res"] = sum(response_counts)/df.shape[1] #add respsonse count
    
    return df

In [17]:
df_18 = pd.DataFrame()
for sheet in CHOSEN_SHEETS:
    temp_df = clean(covid_data, sheet)
    df_18 = df_18.append(temp_df)

In [18]:
df_18

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,...,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,Q27,avg_res
England,88.783,82,85.2945,81.2207,84.0735,84.9266,80.8541,72.5153,72.7355,74.5888,...,86.6403,86.7844,69.0584,83.7235,83.4897,75.496,60.9393,56.4158,83.1474,270038.925926
Scotland,88.9153,82.9435,85.0283,80,83.0284,83.9717,78.0282,71.1981,73.17,64.1133,...,86.8877,87.8069,67.776,85.8327,85.8862,73.9436,54.9438,51.8254,83,24364.407407
Wales,89.9286,83.0357,84.9642,82,83.9285,84.9284,81.9644,75.1072,75.2484,75.1423,...,86.7546,86.6494,71.789,84.931,85.7859,78.9284,63.7505,58.6332,84.9281,16035.259259
Northern Ireland,89.0482,82.0962,85.1927,83.2409,83.1928,85.1929,84.1914,73.3376,75.3857,69.2414,...,87.6688,87.7181,69.9044,87.8098,83.8554,75,58.9525,58.5511,84.0968,7804.962963
